<a href="https://colab.research.google.com/github/nrodman/FoodRecommendation/blob/main/neuralnetwork12recipes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

from google.colab import drive
drive.mount('/content/drive')
data_dir = '/content/drive/My Drive/project1/'
dataset_path = data_dir + 'reduced.csv'
recipes_df = pd.read_csv(dataset_path)

print("Columns in the merged dataset:")
print(recipes_df.columns)

print("Merged Dataset:")
print(recipes_df.head())

# defining feature columns
feature_columns = ['calories', 'fatcontent', 'proteincontent']
recipes_df = recipes_df.dropna(subset=feature_columns)
features = recipes_df[feature_columns]
target = (recipes_df['rating'] >= 4).astype(int)  # Binary classification: 1 for rating >= 4, else 0

# standardizing the features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# data splitting into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, random_state=42)

# sequential neural network model
def build_nn_model():
    model = Sequential()
    model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# training and evaluating the model
def train_evaluate_nn(X_train, y_train, X_test, y_test):
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)

    fold_metrics = []

    for train_index, val_index in kfold.split(X_train):
        X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

        nn_model = build_nn_model()
        nn_model.fit(X_train_fold, y_train_fold, epochs=10, batch_size=32, verbose=0, validation_data=(X_val_fold, y_val_fold))

        y_val_pred = (nn_model.predict(X_val_fold) > 0.5).astype(int)

        accuracy = accuracy_score(y_val_fold, y_val_pred)
        f1 = f1_score(y_val_fold, y_val_pred, average='weighted')
        precision = precision_score(y_val_fold, y_val_pred, average='weighted')
        recall = recall_score(y_val_fold, y_val_pred, average='weighted')

        fold_metrics.append((accuracy, f1, precision, recall))

    # accuracy tests
    nn_model_final = build_nn_model()
    nn_model_final.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
    y_test_pred = (nn_model_final.predict(X_test) > 0.5).astype(int)

    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred, average='weighted')
    test_precision = precision_score(y_test, y_test_pred, average='weighted')
    test_recall = recall_score(y_test, y_test_pred, average='weighted')

    print(f"\nFinal Test Set Evaluation - Accuracy: {test_accuracy}, F1 Score: {test_f1}, Precision: {test_precision}, Recall: {test_recall}")

    return nn_model_final

# training and evaluating the modelc again
nn_model = train_evaluate_nn(X_train, y_train, X_test, y_test)

# similar recipes prediction based on feautures
def get_similar_recipes(recipe_id, top_n=4):
    recipe_features = recipes_df[recipes_df['recipeid'] == recipe_id][feature_columns].values
    recipe_features_scaled = scaler.transform(recipe_features)

    predictions = nn_model.predict(features_scaled)
    recipes_df['predictions'] = predictions

    selected_recipe_prediction = nn_model.predict(recipe_features_scaled)[0][0]
    recipes_df['similarity'] = np.abs(recipes_df['predictions'] - selected_recipe_prediction)

    similar_recipes = recipes_df[recipes_df['recipeid'] != recipe_id].sort_values(by='similarity').drop_duplicates(subset='recipeid').head(top_n)
    return similar_recipes

# our 12 recipe IDs to process
recipe_ids = [
    35568, 45809, 203261, 147551, 43023, 148007, 1005, 211432, 27195, 96210, 16333, 308113
]

# each specific recipe ID printed
for recipe_id in recipe_ids:
    selected_recipe = recipes_df[recipes_df['recipeid'] == recipe_id].iloc[0]
    selected_recipe_name = selected_recipe['name']
    print(f"\nRecipe: {selected_recipe_name} (ID: {recipe_id})")

    # similar recipes based on features
    similar_recipes = get_similar_recipes(recipe_id)

    # printing recommended recipes
    print("Recommended Recipes Based on Similar Features:")
    print(similar_recipes[['recipeid', 'name']].head(4))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Columns in the merged dataset:
Index(['recipeid', 'name', 'author', 'totaltime', 'datepublished',
       'description', 'images', 'recipecategory', 'keywords',
       'aggregatedrating', 'reviewcount', 'calories', 'fatcontent',
       'proteincontent', 'recipeinstructions', 'reviewid', 'reviewer',
       'rating', 'bigcategory'],
      dtype='object')
Merged Dataset:
   recipeid                              name  author totaltime  \
0     79703   Easy Seven Layer Rice Casserole   89831   PT1H35M   
1    284696     Baked Corn and Rice Casserole  128473   PT1H20M   
2    105682  Sugar and Cinnamon Spiced Pecans  131500   PT1H10M   
3     26039  Papa John's Garlic Dipping Sauce   27395      PT4M   
4    448521                  Baked Seitan Log  992845   PT1H35M   

               datepublished  \
0  2003-12-29 20:00:00+00:00   
1  2008-02-07 18:31:00+00:00   
2 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


657/657 [==============================] - 1s 1ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


657/657 [==============================] - 1s 2ms/step


...

1/1 [==============================] - 0s 31ms/step
Recommended Recipes Based on Similar Features:
        recipeid                                    name
120249    280527          Keiko's Spicy Dynamite Edamame
103761    126054            Tomato and Mozzarella Burger
87051     190123  North Carolina Boneless Shoulder Roast
30008      65860       Party-Favorite Crabmeat Au Gratin

Recipe: Jalapeno Poppers (ID: 16333)
  20/4104 [..............................] - ETA: 10s 

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 21ms/step
Recommended Recipes Based on Similar Features:
        recipeid                                name
27174     147275                  Three Onion Spread
8633      350443               Super Easy Naan Bread
112635     41253               Cinnamon Honey Butter
120931    156767  World's Best Braised Green Cabbage

Recipe: Greek Grilled Fish (ID: 308113)
  79/4104 [..............................] - ETA: 5s

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 32ms/step
Recommended Recipes Based on Similar Features:
        recipeid                             name
8114       17298      Challah (Braided Egg Bread)
43744      88989               Easy Blueberry Pie
42016     242405                    Hot Milk Cake
110664    244220  Traditional Irish Guinness Stew
